In [67]:
import yfinance as yf
import backtrader as bt
import datetime

msft = yf.Ticker("NVDA")
# get historical market data
hist = msft.history(period="max")

In [70]:
class TestStrategy(bt.Strategy):
    params={'maperiod':20}

    def log(self, txt, dt=None):
        # logging function. print the date, and then
        # whatever txt is passed to it
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):

        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)
        self.bb1 = bt.indicators.BBands(self.datas[0], period=self.params.maperiod, devfactor=1)
        self.bb2 = bt.indicators.BBands(self.datas[0], period=self.params.maperiod, devfactor=2)

        # Indicators for the plotting show
        #bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        #bt.indicators.WeightedMovingAverage(self.datas[0], period=25, subplot=True)
        #bt.indicators.StochasticSlow(self.datas[0])
        #bt.indicators.MACDHisto(self.datas[0])
        #rsi = bt.indicators.RSI(self.datas[0])
        #bt.indicators.SmoothedMovingAverage(rsi, period=10)
        #bt.indicators.ATR(self.datas[0], plot=False)
        #bt.indicators.BBands(self.datas[0], period=20, devfactor=1)
        #bt.indicators.BBands(self.datas[0], period=20, devfactor=2)
        #bt.indicators.BBands(self.datas[0], period=25,devfactor=3)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                """self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))"""

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                """self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))"""

            self.bar_executed = len(self)

        #elif order.status in [order.Canceled, order.Margin, order.Rejected]:
        #    #self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        #self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
        #         (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        if not self.position: # if not in the market, we can buy
            if self.dataclose[0] > self.bb2.top[0]:
            #if self.dataclose[0] > self.sma[0]:
                #self.log('BUY CREATE, %.2f' % self.dataclose[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
        else:
            if self.dataclose[0] < self.bb1.top[0]:
            #if self.dataclose[0] < self.sma[0]:
                #self.log('SELL CREATE, %.2f' % self.dataclose[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
        

init_cash = 5000.0
testparams = [{"maperiod":n} for n in range(5,30)]

final_scores = []

for p in testparams:
    cerebro = bt.Cerebro()
    
    cerebro.addstrategy(TestStrategy, maperiod=p['maperiod'])

    # Create a Data Feed
    data = bt.feeds.PandasDirectData(dataname=hist,
                                     fromdate=datetime.datetime(2016, 1, 1),
                                     todate=datetime.datetime(2020, 12, 31))
    cerebro.adddata(data)

    cerebro.broker.setcash(init_cash)

    cerebro.addsizer(bt.sizers.FixedSize, stake=100) # number of shares to buy

    print(p)
    print('    Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('    Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    final_scores.append(cerebro.broker.getvalue())

    #cerebro.plot(path="./testplot.png")

{'maperiod': 5}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 5000.00
{'maperiod': 6}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 5737.81
{'maperiod': 7}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 5736.70
{'maperiod': 8}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 5779.69
{'maperiod': 9}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 6050.18
{'maperiod': 10}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 6041.17
{'maperiod': 11}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 6023.49
{'maperiod': 12}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 6063.86
{'maperiod': 13}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 5685.42
{'maperiod': 14}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 5853.00
{'maperiod': 15}
    Starting Portfolio Value: 5000.00
    Final Portfolio Value: 5783.92
{'maperiod': 16

In [69]:
final_scores

[5000.0,
 5245.007648399996,
 4971.443762390614,
 4826.584330192421,
 4952.573691346345,
 5286.318514979361,
 5361.302179402177,
 5364.002796633573,
 4846.767243363942,
 5335.1916951332505,
 5410.537744634214,
 5356.916142142608,
 4936.314303742058,
 5431.131168181951,
 5356.983459061049,
 5352.737248002047,
 5363.745400769029,
 5504.957933484447,
 5351.661469625597,
 5452.338054719009,
 5331.90926502421,
 5411.853839104669,
 5323.288546646262,
 5020.66389707572,
 4974.604059052469]